<a href="https://colab.research.google.com/github/ChadDelany/Drought_Prediction/blob/main/04_Modeling_RAPIDS_selectFeatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Drought Prediction: Modeling**

This notebook was run on Google Colab Pro+.  The RAPIDS system was used to access GPU processing.  Previous attempts to run the models using Pandas with CPUs took 24+ hours to run and often crashed due to exceeding existing resources.  The RAPIDS GPU processing allowed models to run usually within 5 minutes and at a maximum of 15 minutes.  Setting up RAPIDS to run on Google Colab Pro+ takes between 15 minutes to 1 hour depending on resource availability on Google Colab Pro+.

ALL MODELS RERUN WITH SELECTED FEATURES TO DETERMINE VARIABLE IMPORTANCE AND MODEL EXPLAINABILITY.

# Environment Sanity Check #

Click the _Runtime_ dropdown at the top of the page, then _Change Runtime Type_ and confirm the instance type is _GPU_.

Check the output of `!nvidia-smi` to make sure you've been allocated a Tesla T4, P4, or P100.

In [ ]:
!nvidia-smi

Thu Aug 18 15:22:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    31W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#Setup:
This notebook was built on RAPIDS 0.13 stable and is based on this [DataCamp Tutorial](https://www.datacamp.com/community/tutorials/xgboost-in-python).  tested and working on 0.19 stable.

#Setup:
Set up script installs
1. Updates gcc in Colab
1. Installs Conda
1. Install RAPIDS' current stable version of its libraries, as well as some external libraries including:
  1. cuDF
  1. cuML
  1. cuGraph
  1. cuSpatial
  1. cuSignal
  1. BlazingSQL
  1. xgboost
1. Copy RAPIDS .so files into current working directory, a neccessary workaround for RAPIDS+Colab integration.


In [ ]:
!pip install pynvml

# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/env-check.py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 46 kB 3.2 MB/s 
Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 300, done.
remote: Counting objects: 100% (129/129), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 300 (delta 74), reused 99 (delta 55), pack-reused 171
Receiving objects: 100% (300/300), 87.58 KiB | 7.96 MiB/s, done.
Resolving deltas: 100% (136/136), done.
***********************************************************************
Woo! Your instance has the right kind of GPU, a Tesla P100-PCIE-16GB!
***********************************************************************



In [ ]:
# This will update the Colab environment and restart the kernel.  Don't run the next cell until you see the session crash.
!bash rapidsai-csp-utils/colab/update_gcc.sh
import os
os._exit(00)

Updating your Colab environment.  This will restart your kernel.  Don't Panic!
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:7 http://ppa.launchpad.net/ubuntu-toolchain-r/test/ubuntu bionic InRelease [20.8 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [2,095 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com

In [ ]:
# This will install CondaColab.  This will restart your kernel one last time.  Run this cell by itself and only run the next cell once you see the session crash.
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:35
🔁 Restarting kernel...


In [ ]:
# you can now run the rest of the cells as normal
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


In [ ]:
# Installing RAPIDS is now 'python rapidsai-csp-utils/colab/install_rapids.py <release> <packages>'
# The <release> options are 'stable' and 'nightly'.  Leaving it blank or adding any other words will default to stable.
!python rapidsai-csp-utils/colab/install_rapids.py stable
import os
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'
os.environ['CONDA_PREFIX'] = '/usr/local'

Found existing installation: cffi 1.14.5
Uninstalling cffi-1.14.5:
  Successfully uninstalled cffi-1.14.5
Found existing installation: cryptography 3.4.5
Uninstalling cryptography-3.4.5:
  Successfully uninstalled cryptography-3.4.5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Installing RAPIDS Stable 21.12
Starting the RAPIDS install on Colab.  This will take about 15 minutes.
Solving environment: ...working... WARNING conda.core.solve:_add_specs(611): pinned spec cudatoolkit=11.1 conflicts with explicit specs.  Overriding pinned spec.
failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... WARNING conda.core.solve:_add_specs(611): pinned spec cudatoolkit=11.1 conflicts with explicit specs.  Overriding pinned spec.
failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... WARNING conda.core.solve:_add_specs(611): pinned spec c

## Load RAPIDS libraries

In [ ]:
# RAPIDS libraries for accessing GPU processing for running models.  Instead of taking 24+ hours to run models, it only takes 15 minutes or less.
import cudf
import cuml
import cupy

import pandas as pd

import pynvml
import numpy as np


## Load additional Libraries.

In [ ]:
# Import Sklearn metrics.  The RAPIDS metrics currently appeared bugged.
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import cross_val_score, StratifiedKFold

# For displaying model metrics in an easily readable table form.
from IPython.display import HTML, display
import tabulate

# Functions

## Model Accuracy Assessment for Regression.

In [ ]:
# Function for Model Accuracy Assessment for Regression.  Input is in Pandas because RAPIDS metrics currently appeared to be bugged.
def reg_metric(y_test, y_pred):
  # Calculation of metrics
  r2 = str(round(r2_score(y_test, y_pred), 3))
  mse = str(round(mean_squared_error(y_test, y_pred), 3))
  rmse = str(round(np.sqrt(mean_squared_error(y_test, y_pred)), 3))
  mae = str(round(mean_absolute_error(y_test, y_pred), 3))

  #create table for display
  metric = [['Metric', 'Value'],
            ['R**2:', r2],
            ['MSE:', mse],
            ['RMSE:', rmse],
            ['MAE:', mae]]
  table = tabulate.tabulate(metric, tablefmt='html')
  display(HTML(table))

  # Return metric values as strings for later display.
  return(r2, mse, rmse, mae)

## Coefficients from Regression Models.

In [ ]:
# Function to pull coefficients for regression models.
def reg_coefs(model):
  # get coefficients from RAPIDS model
  coefs = model.coef_
  coefs = coefs.to_pandas()

  # associate variable names with coefficients
  features = X_train.columns
  
  # Create Pandas Series with appropriate labels
  coefs = coefs.set_axis(features)

  return(coefs)

## Model Accuracy Assessment for Classification.

In [ ]:
# Function for calculating Classification Metrics
def class_metric(ycat_test, y_pred):
  #Calculation of metrics
  accuracy = str(np.round(cuml.metrics.accuracy.accuracy_score(ycat_test, y_pred), 3) * 100)
  roc_auc = str(np.round(cuml.metrics.roc_auc_score(ycat_test, y_pred), 3))

  cp = np.round(cuml.metrics.confusion_matrix(ycat_test, y_pred, normalize='pred'), 3) * 100
  pred_perclass = [cp[0][0].get(), cp[1][1].get(), cp[2][2].get(), cp[3][3].get(), cp[4][4].get(), cp[5][5].get()]

  cp_mean = str(np.round(np.mean(pred_perclass), 1))
  cp_std = str(np.round(np.std(pred_perclass), 1))

  print(f'Accuracy Score: {accuracy}%')
  print(f'ROC AUC: {roc_auc}')
  print(f'Mean Accuracy per Class & Standard Deviation: {cp_mean}% +/- {cp_std}%')
  print(cp)

  return(accuracy, roc_auc, cp, cp_mean, cp_std)

# Load Dataset.

In [ ]:
# Local location of the data

# Location on Windows
# local_data = 'D:\\Data_Science\\DroughtProject\\Data\\' 

# Location on Linux
# local_data = '/home/chad/Data/Drought_Prediction/' 

# Load local data into Google Colab
# from google.colab import files
# files = files.upload()

In [ ]:
# Accessing Google Drive by mounting it locally
# https://towardsdatascience.com/7-ways-to-load-external-data-into-google-colab-7ba73e7d5fc7
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Location on Google Drive
local_data = '/content/drive/MyDrive/Colab Notebooks/'

In [ ]:
# Load the dataset that contains training (meteorological variables) resampled weekly with mean, max, min
# and the soil variables that have been merged on the county 'fips' value
# This version of the file has already been scaled for the mean equal to zero and the variance to a standard deviation via StandardScaler.

tsm = cudf.read_csv(local_data + 'train_soil_stats_scaled.csv',
                        parse_dates=['date'],
                        index_col=['index'],
                        header=0)

In [ ]:
# Unmount Google Drive.
drive.flush_and_unmount()

## Select Features and Target for Models, Features Reduced due to Collinearity.

In [ ]:
tsm.columns

Index(['fips', 'date', 'score', 'PRECTOT_mean', 'PS_mean', 'QV2M_mean',
       'T2M_mean', 'T2MDEW_mean', 'T2MWET_mean', 'T2M_MAX_mean',
       'T2M_MIN_mean', 'T2M_RANGE_mean', 'TS_mean', 'WS10M_mean',
       'WS10M_MAX_mean', 'WS10M_MIN_mean', 'WS10M_RANGE_mean', 'WS50M_mean',
       'WS50M_MAX_mean', 'WS50M_MIN_mean', 'WS50M_RANGE_mean', 'PRECTOT_max',
       'PS_max', 'QV2M_max', 'T2M_max', 'T2MDEW_max', 'T2MWET_max',
       'T2M_MAX_max', 'T2M_MIN_max', 'T2M_RANGE_max', 'TS_max', 'WS10M_max',
       'WS10M_MAX_max', 'WS10M_MIN_max', 'WS10M_RANGE_max', 'WS50M_max',
       'WS50M_MAX_max', 'WS50M_MIN_max', 'WS50M_RANGE_max', 'PRECTOT_min',
       'PS_min', 'QV2M_min', 'T2M_min', 'T2MDEW_min', 'T2MWET_min',
       'T2M_MAX_min', 'T2M_MIN_min', 'T2M_RANGE_min', 'TS_min', 'WS10M_min',
       'WS10M_MAX_min', 'WS10M_MIN_min', 'WS10M_RANGE_min', 'WS50M_min',
       'WS50M_MAX_min', 'WS50M_MIN_min', 'WS50M_RANGE_min', 'lat', 'lon',
       'elevation', 'slope1', 'slope2', 'slope3', 'slope4

In [ ]:
# Breaking out independent numerical variables from target variable, categorical variable ('fips'), and date.
# cols = tsm.columns.tolist()
features = ['PRECTOT_mean', 'PS_mean', 'QV2M_min', 'T2M_MAX_max', 
            'WS10M_mean', 'T2M_RANGE_mean', 'WS10M_RANGE_mean', 
            'WS50M_RANGE_mean', 'PRECTOT_max', 'PS_max', 'T2M_RANGE_max', 
            'PRECTOT_min', 'PS_min', 'WS10M_RANGE_min', 'WS50M_RANGE_min', 
            'lat', 'lon','elevation', 'slope1', 'slope2', 'slope3', 'slope4',
            'slope5', 'slope6', 'slope7', 'slope8', 'aspectN', 'aspectE', 
            'aspectS', 'aspectW', 'WAT_LAND', 'NVG_LAND', 'URB_LAND', 'GRS_LAND', 
            'FOR_LAND', 'CULTRF_LAND', 'CULTIR_LAND', 'SQ1', 'SQ3', 'SQ4', 'SQ6']

# Separating out the features
X = tsm[features]

# Separating out the target
y = tsm[['score']]

In [ ]:
# Converting 'y' from Panda Dataframe to Panda Series to avoid conflicts with type when running RAPIDS models.
y = y['score']

# Convert X from float64 to float32 in order to utilize GPU processing instead of CPU processing.  RAPIDS currently does not support float64.
X = X.astype('float32')

# Convert y from float64 to float32 in order to utilize GPU processing instead of CPU processing.
y = y.astype('float32')

# Create target for classication models.  Drought Score was originally an integer class ranging 0 - 5.
y_cat = np.round(y,0)
y_cat = y_cat.astype(int)

## Train / Test Split, Random_State=42

In [ ]:
# Create first train/test split for regression.  Automated folds from cross-validation are consuming too many resources and taking too long to process.
# The datset is significantly skewed with many examples near 0 and few examples near 5.  The Train/Test split is therefore stratified to account for this.
X_train, X_test, y_train, y_test = cuml.model_selection.train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42, stratify=y_cat)

# Convert y_test to Pandas for metrics calc.
y_test = y_test.to_pandas()

# Not-Even-A-Model

In [ ]:
# Using sklearn requires Pandas & not RAPIDS cudf
X_train = X_train.to_pandas()
X_test = X_test.to_pandas()
y_train = y_train.to_pandas()
# y_test = y_test.to_pandas()

# Dummy Regressor to establish a baseline.
Dumb_reg = DummyRegressor(strategy='mean')
Dumb_reg.fit(X_train, y_train)
y_pred = Dumb_reg.predict(X_test)

In [ ]:
# Dummy Regresion Model assessment: R-squared, Mean Squared Error, Root MSE, Mean Absolute Error.
Dum_r2, Dum_mse, Dum_rmse, Dum_mae = reg_metric(y_test, y_pred)

Metric,Value
R**2:,0.622
MSE:,0.567
RMSE:,0.753
MAE:,0.52


In [ ]:
# There is a bug in RAPIDS for converting Pandas Series back into cudf Series, so just rerunning the Test / Train split with the same random state.
X_train, X_test, y_train, y_test = cuml.model_selection.train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42, stratify=y_cat)

# Convert y_test to Pandas for metrics calc.
y_test = y_test.to_pandas()

# Regression Models

## Linear Regression

In [ ]:
# Linear Regression 
Lin_model = cuml.LinearRegression(fit_intercept=True, normalize=True)
Lin_model.fit(X_train, y_train)
y_pred = Lin_model.predict(X_test)

In [ ]:
# Convert RAPIDS to Pandas for Regression Metrics Calculations.
y_pred = y_pred.to_pandas()

In [ ]:
# Linear Regresion Model assessment: R-squared, Mean Squared Error, Root MSE, Mean Absolute Error.
Lin_r2, Lin_mse, Lin_rmse, Lin_mae = reg_metric(y_test, y_pred)

Metric,Value
R**2:,0.157
MSE:,1.262
RMSE:,1.123
MAE:,0.858


In [ ]:
# Looking at coefficients for the regression model to determine importance of variables within the model.  
# These results are only preliminary and need to assessed when there is less correlation between input variables.

# Function call to pull coefficients.
coefs = reg_coefs(Lin_model)

# Display sorted values of coefficients.
coefs.sort_values(ascending=False)

PS_min              0.748815
WS10M_RANGE_mean    0.279152
T2M_MAX_max         0.260122
T2M_RANGE_mean      0.138986
slope4              0.133555
SQ1                 0.129967
slope3              0.126802
slope6              0.095450
slope7              0.057758
PRECTOT_min         0.030386
slope8              0.018832
WS10M_RANGE_min     0.004437
SQ3                 0.000496
slope1             -0.002613
CULTIR_LAND        -0.003659
PRECTOT_mean       -0.007030
SQ4                -0.007562
WS50M_RANGE_min    -0.007980
WAT_LAND           -0.011483
elevation          -0.014892
T2M_RANGE_max      -0.020142
GRS_LAND           -0.022597
slope5             -0.023317
URB_LAND           -0.034839
NVG_LAND           -0.037071
slope2             -0.050015
WS10M_mean         -0.052763
aspectE            -0.067856
aspectS            -0.068648
aspectW            -0.079802
aspectN            -0.085419
SQ6                -0.092158
PRECTOT_max        -0.092593
FOR_LAND           -0.113713
lon           

In [ ]:
# Displaying all coefficient values to understand range and variability.
# Sorted absolute values to understand total influence on model.
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(abs(coefs).sort_values(ascending=False))

PS_min              0.748815
PS_mean             0.682575
QV2M_min            0.280689
WS10M_RANGE_mean    0.279152
T2M_MAX_max         0.260122
WS50M_RANGE_mean    0.200847
PS_max              0.178124
lat                 0.142025
T2M_RANGE_mean      0.138986
slope4              0.133555
SQ1                 0.129967
slope3              0.126802
CULTRF_LAND         0.125314
lon                 0.124503
FOR_LAND            0.113713
slope6              0.095450
PRECTOT_max         0.092593
SQ6                 0.092158
aspectN             0.085419
aspectW             0.079802
aspectS             0.068648
aspectE             0.067856
slope7              0.057758
WS10M_mean          0.052763
slope2              0.050015
NVG_LAND            0.037071
URB_LAND            0.034839
PRECTOT_min         0.030386
slope5              0.023317
GRS_LAND            0.022597
T2M_RANGE_max       0.020142
slope8              0.018832
elevation           0.014892
WAT_LAND            0.011483
WS50M_RANGE_mi

In [ ]:
# The variables with the largest coefficients from the simple linear regresssion.
top_features = abs(coefs).sort_values(ascending=False)
top_features[:15]

PS_min              0.748815
PS_mean             0.682575
QV2M_min            0.280689
WS10M_RANGE_mean    0.279152
T2M_MAX_max         0.260122
WS50M_RANGE_mean    0.200847
PS_max              0.178124
lat                 0.142025
T2M_RANGE_mean      0.138986
slope4              0.133555
SQ1                 0.129967
slope3              0.126802
CULTRF_LAND         0.125314
lon                 0.124503
FOR_LAND            0.113713
dtype: float32

### Ridge Regresssion

In [ ]:
# Ridge Regression - L2 regularization
Ridge_model = cuml.Ridge(fit_intercept=True, normalize=True)
Ridge_model.fit(X_train, y_train)
y_pred = Ridge_model.predict(X_test)

In [ ]:
# Convert y_pred to Pandas for metrics calc.
y_pred = y_pred.to_pandas()

In [ ]:
# Regresion Model assessment: R-squared, Mean Squared Error, Root MSE, Mean Absolute Error.
Ridge_r2, Ridge_mse, Ridge_rmse, Ridge_mae = reg_metric(y_test, y_pred)

Metric,Value
R**2:,0.13
MSE:,1.303
RMSE:,1.142
MAE:,0.888


In [ ]:
# Pulling coefficients from the model.
coefs = reg_coefs(Ridge_model)
coefs.sort_values(ascending=False)

T2M_RANGE_mean      0.083155
T2M_MAX_max         0.064645
T2M_RANGE_max       0.060246
GRS_LAND            0.046142
elevation           0.023435
CULTIR_LAND         0.022852
WS10M_RANGE_min     0.021058
SQ1                 0.019350
NVG_LAND            0.018064
WS10M_RANGE_mean    0.014902
slope3              0.014718
WAT_LAND            0.003518
PRECTOT_min         0.003160
slope7              0.002333
slope8              0.001277
slope1              0.000460
WS10M_mean         -0.000292
slope4             -0.000375
aspectS            -0.000447
SQ3                -0.000571
WS50M_RANGE_min    -0.000985
aspectE            -0.001067
aspectW            -0.001547
SQ6                -0.003963
aspectN            -0.004043
slope2             -0.005319
WS50M_RANGE_mean   -0.005369
SQ4                -0.006236
URB_LAND           -0.008621
slope6             -0.011999
FOR_LAND           -0.013874
PS_min             -0.020828
slope5             -0.023223
PS_mean            -0.024865
QV2M_min      

In [ ]:
# The variables with the largest coefficients from the Ridge regresssion.
top_features = abs(coefs).sort_values(ascending=False)
top_features[:15]

T2M_RANGE_mean    0.083155
lat               0.076267
T2M_MAX_max       0.064645
T2M_RANGE_max     0.060246
lon               0.058942
GRS_LAND          0.046142
PRECTOT_max       0.042539
CULTRF_LAND       0.040739
PRECTOT_mean      0.039206
PS_max            0.027385
QV2M_min          0.026084
PS_mean           0.024865
elevation         0.023435
slope5            0.023223
CULTIR_LAND       0.022852
dtype: float32

### Lasso Regression

In [ ]:
# Lasso Regression - L1 regularization
Lasso_model = cuml.Lasso(fit_intercept=True, normalize=True)
Lasso_model.fit(X_train, y_train)
y_pred = Lasso_model.predict(X_test)

In [ ]:
# Convert y_pred to Pandas for metrics calc.
y_pred = y_pred.to_pandas()

In [ ]:
# Regresion Model assessment: R-squared, Mean Squared Error, Root MSE, Mean Absolute Error.
Lasso_r2, Lasso_mse, Lasso_rmse, Lasso_mae = reg_metric(y_test, y_pred)

Metric,Value
R**2:,-0.0
MSE:,1.497
RMSE:,1.224
MAE:,0.974


In [ ]:
# Pulling coefficients from the model.
coefs = reg_coefs(Lasso_model)
coefs.sort_values(ascending=False)

PRECTOT_mean        0.0
slope4              0.0
slope6              0.0
slope7              0.0
slope8              0.0
aspectN             0.0
aspectE             0.0
aspectS             0.0
aspectW             0.0
WAT_LAND            0.0
NVG_LAND            0.0
URB_LAND            0.0
GRS_LAND            0.0
FOR_LAND            0.0
CULTRF_LAND         0.0
CULTIR_LAND         0.0
SQ1                 0.0
SQ3                 0.0
SQ4                 0.0
slope5              0.0
slope3              0.0
PS_mean             0.0
slope2              0.0
QV2M_min            0.0
T2M_MAX_max         0.0
WS10M_mean          0.0
T2M_RANGE_mean      0.0
WS10M_RANGE_mean    0.0
WS50M_RANGE_mean    0.0
PRECTOT_max         0.0
PS_max              0.0
T2M_RANGE_max       0.0
PRECTOT_min         0.0
PS_min              0.0
WS10M_RANGE_min     0.0
WS50M_RANGE_min     0.0
lat                 0.0
lon                 0.0
elevation           0.0
slope1              0.0
SQ6                 0.0
dtype: float32

In [ ]:
# The variables with the largest coefficients from the simple linear regresssion.
top_features = abs(coefs).sort_values(ascending=False)
top_features[:15]

PRECTOT_mean    0.0
slope4          0.0
slope6          0.0
slope7          0.0
slope8          0.0
aspectN         0.0
aspectE         0.0
aspectS         0.0
aspectW         0.0
WAT_LAND        0.0
NVG_LAND        0.0
URB_LAND        0.0
GRS_LAND        0.0
FOR_LAND        0.0
CULTRF_LAND     0.0
dtype: float32

### ElasticNet Regression

In [ ]:
# ElasticNet Regression - combination of L1 and L2 regularization
EN_model = cuml.ElasticNet(alpha=0.33)
EN_model.fit(X_train, y_train)
y_pred = EN_model.predict(X_test)

In [ ]:
# Convert y_pred to Pandas for metrics calc.
y_pred = y_pred.to_pandas()

In [ ]:
# Regresion Model assessment: R-squared, Mean Squared Error, Root MSE, Mean Absolute Error.
EN_r2, EN_mse, EN_rmse, EN_mae = reg_metric(y_test, y_pred)

Metric,Value
R**2:,0.074
MSE:,1.387
RMSE:,1.178
MAE:,0.929


In [ ]:
# Pulling coefficients from the model.
coefs = reg_coefs(EN_model)
coefs.sort_values(ascending=False)

T2M_RANGE_mean      0.145189
GRS_LAND            0.037953
PRECTOT_mean        0.000000
WAT_LAND            0.000000
slope6              0.000000
slope7              0.000000
slope8              0.000000
aspectN             0.000000
aspectE             0.000000
aspectS             0.000000
aspectW             0.000000
URB_LAND            0.000000
NVG_LAND            0.000000
slope4              0.000000
FOR_LAND            0.000000
CULTRF_LAND         0.000000
CULTIR_LAND         0.000000
SQ1                 0.000000
SQ3                 0.000000
SQ4                 0.000000
slope5              0.000000
slope3              0.000000
PS_mean             0.000000
T2M_RANGE_max       0.000000
QV2M_min            0.000000
T2M_MAX_max         0.000000
WS10M_mean          0.000000
WS10M_RANGE_mean    0.000000
WS50M_RANGE_mean    0.000000
PRECTOT_max         0.000000
PS_max              0.000000
PRECTOT_min         0.000000
slope2              0.000000
PS_min              0.000000
WS10M_RANGE_mi

In [ ]:
# The variables with the largest coefficients from the simple linear regresssion.
top_features = abs(coefs).sort_values(ascending=False)
top_features[:15]

T2M_RANGE_mean    0.145189
GRS_LAND          0.037953
lon               0.033884
PRECTOT_mean      0.000000
WAT_LAND          0.000000
slope7            0.000000
slope8            0.000000
aspectN           0.000000
aspectE           0.000000
aspectS           0.000000
aspectW           0.000000
URB_LAND          0.000000
NVG_LAND          0.000000
slope5            0.000000
FOR_LAND          0.000000
dtype: float32

## Nearest Neighbors Regresssion

In [ ]:
NNreg_model = cuml.neighbors.KNeighborsRegressor()
NNreg_model.fit(X_train, y_train)
y_pred = NNreg_model.predict(X_test)

In [ ]:
# Convert y_pred to Pandas for metrics calc.
y_pred = y_pred.to_pandas()

In [ ]:
# Regresion Model assessment: R-squared, Mean Squared Error, Root MSE, Mean Absolute Error.
NN_r2, NN_mse, NN_rmse, NN_mae = reg_metric(y_test, y_pred)

Metric,Value
R**2:,0.188
MSE:,1.216
RMSE:,1.103
MAE:,0.776


### Nearest Neighbor Hyperparameter Search

In [ ]:
# Convert y_test to pandas if it has not already been converted.
# y_test = y_test.to_pandas()

In [ ]:
# Search for best n_neighbors hyperparameter.  Model parameters are not causing the system to crash.
r2_list=[]
mse_list=[]
rmse_list=[]
mae_list=[]

# Searching nearest neighbors from 2 to 9.
for i in range(8):
  NNreg_model = cuml.neighbors.KNeighborsRegressor(n_neighbors=(i+2))
  NNreg_model.fit(X_train, y_train)
  y_pred = NNreg_model.predict(X_test)

  # Convert y_pred to Pandas for metrics calc.
  y_pred = y_pred.to_pandas()

  # Call custom metric function
  print(f'Nearest Neighbor: {i+2}')
  NN_r2, NN_mse, NN_rmse, NN_mae = reg_metric(y_test, y_pred)

  r2_list.append(NN_r2)
  mse_list.append(NN_mse)
  rmse_list.append(NN_rmse)
  mae_list.append(NN_mae)

Nearest Neighbor: 2


Metric,Value
R**2:,-0.001
MSE:,1.499
RMSE:,1.224
MAE:,0.809


Nearest Neighbor: 3


Metric,Value
R**2:,0.106
MSE:,1.338
RMSE:,1.157
MAE:,0.789


Nearest Neighbor: 4


...

Metric,Value
R**2:,0.219
MSE:,1.169
RMSE:,1.081
MAE:,0.773


Nearest Neighbor: 8


Metric,Value
R**2:,0.228
MSE:,1.155
RMSE:,1.075
MAE:,0.773


Nearest Neighbor: 9


Metric,Value
R**2:,0.235
MSE:,1.146
RMSE:,1.07
MAE:,0.773


In [ ]:
# Print Accuracy Score for each n_neighbors
for i in range(8):
  print(f'n_neighbors={(i+2)}, Mean Absolute Error: {mae_list[i]}')

n_neighbors=2, Mean Absolute Error: 0.809
n_neighbors=3, Mean Absolute Error: 0.789
n_neighbors=4, Mean Absolute Error: 0.78
n_neighbors=5, Mean Absolute Error: 0.776
n_neighbors=6, Mean Absolute Error: 0.774
n_neighbors=7, Mean Absolute Error: 0.773
n_neighbors=8, Mean Absolute Error: 0.773
n_neighbors=9, Mean Absolute Error: 0.773


The Mean Absolute Error never goes below 0.5.  Since each integer value correlates to a Drought Score Category, having an error that exceeds 0.5 is not practically useful.

## Random Forest Regression Model

In [ ]:
# Random Forest Regression model, default max_depth=16, default n_estimators=100.
RFreg_model = cuml.ensemble.RandomForestRegressor()
RFreg_model.fit(X_train, y_train)
y_pred = RFreg_model.predict(X_test)

In [ ]:
# Convert y_pred to Pandas for metrics calc.
y_pred = y_pred.to_pandas()

In [ ]:
# Regresion Model assessment: R-squared, Mean Squared Error, Root MSE, Mean Absolute Error.
RF_r2, RF_mse, RF_rmse, RF_mae = reg_metric(y_test, y_pred)

Metric,Value
R**2:,0.415
MSE:,0.877
RMSE:,0.936
MAE:,0.694


### Random Forest Regression, max_depth = 100

In [ ]:
# Random Forest Regression model: MAX DEPTH = 100.
RFreg_model = cuml.ensemble.RandomForestRegressor(max_depth=100)
RFreg_model.fit(X_train, y_train)
y_pred = RFreg_model.predict(X_test)

In [ ]:
# Convert y_pred to Pandas for metrics calc.
y_pred = y_pred.to_pandas()

In [ ]:
# Regresion Model assessment: R-squared, Mean Squared Error, Root MSE, Mean Absolute Error.
RF_r2, RF_mse, RF_rmse, RF_mae = reg_metric(y_test, y_pred)

Metric,Value
R**2:,0.618
MSE:,0.573
RMSE:,0.757
MAE:,0.521


### Random Forest Regression, max_depth=100, n_estimators=300

In [ ]:
# Random Forest Regression model: MAX DEPTH = 100.
RFreg_model = cuml.ensemble.RandomForestRegressor(max_depth=100, n_estimators=300)
RFreg_model.fit(X_train, y_train)
y_pred = RFreg_model.predict(X_test)

In [ ]:
# Convert y_pred to Pandas for metrics calc.
y_pred = y_pred.to_pandas()

In [ ]:
# Regresion Model assessment: R-squared, Mean Squared Error, Root MSE, Mean Absolute Error.
RF_r2, RF_mse, RF_rmse, RF_mae = reg_metric(y_test, y_pred)

Metric,Value
R**2:,0.622
MSE:,0.567
RMSE:,0.753
MAE:,0.52


# **Regression Model Summary**.

In [ ]:
# Table Summarizing metrics from the different regression models.
metric = [['Metric', 'Dummy Reg', 'Linear Reg', 'Ridge Reg', 'Lasso Reg', 'ElasticNet', 'Nearest Neighbor', 'Random Forest'],
         ['R**2', Dum_r2, Lin_r2, Ridge_r2, Lasso_r2, EN_r2, NN_r2, RF_r2],
         ['MSE', Dum_mse, Lin_mse, Ridge_mse, Lasso_mse, EN_mse, NN_mse, RF_mse],
         ['RMSE', Dum_rmse, Lin_rmse, Ridge_rmse, Lasso_rmse, EN_rmse, NN_rmse, RF_rmse],
         ['MAE', Dum_mae, Lin_mae, Ridge_mae, Lasso_mae, EN_mae, NN_mae, RF_mae]]
table = tabulate.tabulate(metric, tablefmt='html')

display(HTML(table))

Metric,Dummy Reg,Linear Reg,Ridge Reg,Lasso Reg,ElasticNet,Nearest Neighbor,Random Forest
R**2,0.622,0.157,0.13,-0.0,0.074,0.188,0.622
MSE,0.567,1.262,1.303,1.497,1.387,1.216,0.567
RMSE,0.753,1.123,1.142,1.224,1.178,1.103,0.753
MAE,0.52,0.858,0.888,0.974,0.929,0.776,0.52


Attempts to derive feature importance or understand variable contribution for Random Forest and Nearest Neighbor failed due to exceeding available resources: cuML KernelExplainer, cuML PermutationExplainer, sklearn .feature_importances_, and cuML .get_detailed_text().  So, while Random Forest appears to be the most accurate model for regression, it is not possible with the currently available resources to understand and rank feature importance.

The default values for Lasso Regression (L1 regularization) create a model that appears to simply follow the mean values.  

The errors from the regression models appear to exceed practical usefulness.  Converting to a classification problem will more likely yeild results that are more useful.

# Classification Models

In [ ]:
# Create first train/test split for classification.  Automated folds from cross-validation are consuming too many resources and taking too long to process.
Xcat_train, Xcat_test, ycat_train, ycat_test = cuml.model_selection.train_test_split(X, y_cat, test_size=0.2, shuffle=True, random_state=42, stratify=y_cat)

## Logistic Regression Classification Model

In [ ]:
# Logistic Regression does not converge with default values.
# Logistic Regression (defaults are: fit_intercept=True, C=1, penalty='l2', max_iter=1000, tol=1e-4).
Log_model = cuml.LogisticRegression(fit_intercept=True, C=1, penalty='none', max_iter=10000, tol=5e-4)
Log_model.fit(Xcat_train, ycat_train)
y_pred = Log_model.predict(Xcat_test)

In [ ]:
Log_acc, Log_roc, Log_cp, Log_cpMean, Log_cpSTD = class_metric(ycat_test, y_pred)

Accuracy Score: 59.8%
ROC AUC: 0.631
Mean Accuracy per Class & Standard Deviation: 26.3% +/- 15.8%
[[61.1 30.1 28.4 22.8 18.4 17.9]
 [16.9 22.2 16.8 16.8 16.1 18.1]
 [10.4 17.4 20.2 20.1 15.  16. ]
 [ 6.7 15.7 18.9 21.4 19.1 21.1]
 [ 3.5 12.  10.  13.1 19.  13.2]
 [ 1.3  2.6  5.7  5.7 12.5 13.7]]


## Nearest Neighbors Classification Model

In [ ]:
# Nearest Neighbors Classifier with default values: n_neighbors=5
NNclass_model = cuml.neighbors.KNeighborsClassifier()
NNclass_model.fit(Xcat_train, ycat_train)
y_pred = NNclass_model.predict(Xcat_test)

In [ ]:
NN_acc, NN_roc, NN_cp, NN_cpMean, NN_cpSTD = class_metric(ycat_test, y_pred)

Accuracy Score: 59.4%
ROC AUC: 1.029
Mean Accuracy per Class & Standard Deviation: 33.6% +/- 15.2%
[[67.1 36.8 27.1 25.  22.8 20.8]
 [15.3 30.4 18.2 15.3 14.8 13.2]
 [ 8.7 15.2 28.4 17.  14.  15.7]
 [ 5.4  9.7 14.9 27.3 15.7 14.6]
 [ 2.6  5.6  8.2 11.6 26.  13.4]
 [ 0.9  2.2  3.2  3.9  6.6 22.4]]


### Nearest Neighbor Classification, n_neighbors=10

In [ ]:
# Nearest Neighbors Classifier, n_neighbors=10
NNclass_model = cuml.neighbors.KNeighborsClassifier(n_neighbors=10)
NNclass_model.fit(Xcat_train, ycat_train)
y_pred = NNclass_model.predict(Xcat_test)

In [ ]:
NN_acc, NN_roc, NN_cp, NN_cpMean, NN_cpSTD = class_metric(ycat_test, y_pred)

Accuracy Score: 60.6%
ROC AUC: 0.976
Mean Accuracy per Class & Standard Deviation: 35.8% +/- 13.5%
[[65.9 30.9 22.8 21.6 19.  16. ]
 [15.7 31.6 18.4 14.2 14.4 13.2]
 [ 9.  16.9 30.2 17.4 13.5 14.7]
 [ 5.6 11.2 15.6 30.3 16.7 15.8]
 [ 2.8  6.8  9.2 12.3 29.1 12.9]
 [ 1.   2.7  3.8  4.3  7.4 27.5]]


### Nearest Neighbor Hyperparameter Search

In [ ]:
# Search for best n_neighbors hyperparameter.  Model parameters are not causing the system to crash.
accuracy_list=[]

# Searching Nearest Neighbors from 2 to 9.
for i in range(8):
  NNclass_model = cuml.neighbors.KNeighborsClassifier(n_neighbors=(i+2))
  NNclass_model.fit(Xcat_train, ycat_train)
  y_pred = NNclass_model.predict(Xcat_test)
  score = str(np.round(cuml.metrics.accuracy.accuracy_score(ycat_test, y_pred), 3) * 100)
  accuracy_list.append(score)

In [ ]:
# Print Accuracy Score for each n_neighbors
for i in range(8):
  print(f'n_neighbors={(i+2)}, accuracy score: {accuracy_list[i]}%')

n_neighbors=2, accuracy score: 58.599999999999994%
n_neighbors=3, accuracy score: 58.3%
n_neighbors=4, accuracy score: 59.099999999999994%
n_neighbors=5, accuracy score: 59.4%
n_neighbors=6, accuracy score: 59.9%
n_neighbors=7, accuracy score: 60.099999999999994%
n_neighbors=8, accuracy score: 60.3%
n_neighbors=9, accuracy score: 60.5%


**SUMMARY** of NN: In a survey of n_neighbors from 2 to 10, the accuracy score peaked at n_neighbors=5 (default) with an overeall accuracy score of 68.3%.

## Random Forest Classification Model

In [ ]:
# Default RF model: n_estimators=100, max_depth=16
RFclass_model = cuml.ensemble.RandomForestClassifier()
RFclass_model.fit(Xcat_train, ycat_train)
y_pred = RFclass_model.predict(Xcat_test)
y_pred = y_pred.astype(int)

In [ ]:
# Accuracy Assessment for Classification Model.
RF_acc, RF_roc, RF_cp, RF_cpMean, RF_cpSTD = class_metric(ycat_test, y_pred)

Accuracy Score: 62.4%
ROC AUC: 0.825
Mean Accuracy per Class & Standard Deviation: 55.4% +/- 10.2%
[[63.4 12.7  9.7  7.7  3.1  2.2]
 [16.4 46.9 14.   8.3  3.9  4.5]
 [ 9.8 17.7 42.8 15.4  5.3  5.3]
 [ 6.2 11.5 18.1 47.2 12.7  6.6]
 [ 3.1  8.  10.6 16.2 62.3 11.5]
 [ 1.1  3.1  4.8  5.4 12.7 69.9]]


In [ ]:
# Random Forest Classifier Model, MAX_DEPTH = 100
RFclass_model = cuml.ensemble.RandomForestClassifier(max_depth=100)
RFclass_model.fit(Xcat_train, ycat_train)
y_pred = RFclass_model.predict(Xcat_test)
y_pred = y_pred.astype(int)

In [ ]:
# Accuracy Assessment for Classification Model.
RF_acc, RF_roc, RF_cp, RF_cpMean, RF_cpSTD = class_metric(ycat_test, y_pred)

Accuracy Score: 68.5%
ROC AUC: 1.263
Mean Accuracy per Class & Standard Deviation: 63.6% +/- 6.5%
[[70.4 13.7  7.8  7.1  4.4  2.6]
 [14.  59.1 13.5  6.6  4.2  3.3]
 [ 7.7 14.5 56.8 14.5  5.7  4. ]
 [ 4.7  6.8 14.4 57.5 13.8  5.3]
 [ 2.4  4.3  5.5 11.7 63.7 10.9]
 [ 0.8  1.5  2.   2.6  8.1 73.8]]


#### Random Forest Classifier, max_depth=100, n_estimators=200

In [ ]:
# Random Forest Classifier Model, MAX_DEPTH = 100, n_estimators = 200
RFclass_model = cuml.ensemble.RandomForestClassifier(max_depth=100, n_estimators=200)
RFclass_model.fit(Xcat_train, ycat_train)
y_pred = RFclass_model.predict(Xcat_test)
y_pred = y_pred.astype(int)

In [ ]:
# Accuracy Assessment for Classification Model.
RF_acc, RF_roc, RF_cp, RF_cpMean, RF_cpSTD = class_metric(ycat_test, y_pred)

Accuracy Score: 68.7%
ROC AUC: 1.256
Mean Accuracy per Class & Standard Deviation: 64.8% +/- 6.2%
[[70.3 12.7  7.3  6.7  3.8  2.5]
 [14.  61.  13.   6.3  3.9  2.5]
 [ 7.7 14.3 58.3 14.2  5.   3.9]
 [ 4.8  6.5 14.2 58.5 13.5  4.9]
 [ 2.4  4.1  5.1 11.7 65.9 11.2]
 [ 0.8  1.4  2.   2.5  7.9 75. ]]


#### Random Forest Classifier, max_depth=100, n_estimators=50

In [ ]:
# Random Forest Classifier Model, MAX_DEPTH = 100, n_estimators = 50
RFclass_model = cuml.ensemble.RandomForestClassifier(max_depth=100, n_estimators=50)
RFclass_model.fit(Xcat_train, ycat_train)
y_pred = RFclass_model.predict(Xcat_test)
y_pred = y_pred.astype(int)

In [ ]:
# Accuracy Assessment for Classification Model.
RF_acc, RF_roc, RF_cp, RF_cpMean, RF_cpSTD = class_metric(ycat_test, y_pred)

Accuracy Score: 68.0%
ROC AUC: 1.269
Mean Accuracy per Class & Standard Deviation: 61.3% +/- 7.0%
[[70.5 15.2  8.6  8.   5.5  3.2]
 [14.  56.3 14.   7.6  5.3  4.2]
 [ 7.7 15.1 54.5 14.5  6.5  4.5]
 [ 4.7  7.3 14.8 55.2 14.4  5.5]
 [ 2.3  4.5  5.8 12.1 60.2 11.4]
 [ 0.8  1.6  2.3  2.6  8.1 71.2]]


#### Random Forest Classifier, max_depth=16(default), n_estimators=300

In [ ]:
# Random Forest Classifier Model, MAX_DEPTH = 16, n_estimators = 300
RFclass_model = cuml.ensemble.RandomForestClassifier(max_depth=16, n_estimators=300)
RFclass_model.fit(Xcat_train, ycat_train)
y_pred = RFclass_model.predict(Xcat_test)
y_pred = y_pred.astype(int)

In [ ]:
# Accuracy Assessment for Classification Model.
RF_acc, RF_roc, RF_cp, RF_cpMean, RF_cpSTD = class_metric(ycat_test, y_pred)

Accuracy Score: 62.4%
ROC AUC: 0.815
Mean Accuracy per Class & Standard Deviation: 56.3% +/- 10.3%
[[63.3 12.2  9.4  7.5  3.1  2.1]
 [16.4 48.1 13.7  8.   4.   4. ]
 [ 9.8 17.7 43.3 15.1  4.5  4.4]
 [ 6.2 11.1 18.3 47.9 12.   6.5]
 [ 3.2  7.8 10.6 16.2 64.1 11.8]
 [ 1.1  3.   4.7  5.3 12.2 71.2]]


#### Random Forest Classifier, max_depth=100, n_estimators=300

In [ ]:
# Random Forest Classifier Model, MAX_DEPTH = 100, n_estimators = 300
RFclass_model = cuml.ensemble.RandomForestClassifier(max_depth=100, n_estimators=300)
RFclass_model.fit(Xcat_train, ycat_train)
y_pred = RFclass_model.predict(Xcat_test)
y_pred = y_pred.astype(int)

In [ ]:
# Accuracy Assessment for Classification Model.
RF_acc, RF_roc, RF_cp, RF_cpMean, RF_cpSTD = class_metric(ycat_test, y_pred)

Accuracy Score: 68.8%
ROC AUC: 1.253
Mean Accuracy per Class & Standard Deviation: 65.4% +/- 6.3%
[[70.3 12.7  7.1  6.4  3.8  2.3]
 [14.  61.5 13.   6.2  3.9  2.3]
 [ 7.7 14.2 58.9 14.1  5.   3.5]
 [ 4.8  6.2 14.1 59.1 13.4  4.6]
 [ 2.4  4.   5.  11.6 66.2 11.1]
 [ 0.8  1.4  1.9  2.5  7.7 76.2]]


#### Random Forest Classifier, max_depth=100, n_estimators=350

In [ ]:
# CAUSING SYSTEM TO CRASH, EXCEEDING RAM AVAILABILITY TODAY.
# Random Forest Classifier Model, MAX_DEPTH = 100, n_estimators = 350
RFclass_model = cuml.ensemble.RandomForestClassifier(max_depth=100, n_estimators=350)
RFclass_model.fit(Xcat_train, ycat_train)
y_pred = RFclass_model.predict(Xcat_test)
y_pred = y_pred.astype(int)

In [ ]:
# Accuracy Assessment for Classification Model.
RF_acc, RF_roc, RF_cp, RF_cpMean, RF_cpSTD = class_metric(ycat_test, y_pred)

#### Random Forest Classifier, max_depth=150, n_estimators=300

In [ ]:
# Random Forest Classifier Model, MAX_DEPTH = 150, n_estimators = 300
RFclass_model = cuml.ensemble.RandomForestClassifier(max_depth=150, n_estimators=300)
RFclass_model.fit(Xcat_train, ycat_train)
y_pred = RFclass_model.predict(Xcat_test)
y_pred = y_pred.astype(int)

In [ ]:
# Accuracy Assessment for Classification Model.
RF_acc, RF_roc, RF_cp, RF_cpMean, RF_cpSTD = class_metric(ycat_test, y_pred)

#### Random Forest Classifier: max_depth=200, n_estimators=300, split_criterion=gini(default)

In [ ]:
# Random Forest model: split_criterion=entropy
RFclass_model = cuml.ensemble.RandomForestClassifier(max_depth=200, n_estimators=300, split_criterion=1)
RFclass_model.fit(Xcat_train, ycat_train)
y_pred = RFclass_model.predict(Xcat_test)
y_pred = y_pred.astype(int)

In [ ]:
# Accuracy Assessment for Classification Model.
RF_acc, RF_roc, RF_cp, RF_cpMean, RF_cpSTD = class_metric(ycat_test, y_pred)

#### Random Forest Classifier, max_depth=200, n_estimators=300, split_criterion=entropy

In [ ]:
# Random Forest model: split_criterion=entropy
RFclass_model = cuml.ensemble.RandomForestClassifier(max_depth=200, n_estimators=300, split_criterion=1)
RFclass_model.fit(Xcat_train, ycat_train)
y_pred = RFclass_model.predict(Xcat_test)
y_pred = y_pred.astype(int)

In [ ]:
# Accuracy Assessment for Classification Model.
RF_acc, RF_roc, RF_cp, RF_cpMean, RF_cpSTD = class_metric(ycat_test, y_pred)

# Cross Validation of Optimized Random Forest Model.

## Multiple Random States Validation


In [ ]:
# STILL CRASHING SYSTEM ON THIRD ITERATION (EXCEEDS AVAILABLE RAM) EVEN WITH REDUCED FEATURES.

# Optimal Random Forest model within the processing and resource constraints is max_depth=100, n_estimators=300
# Confirming results through multiple iterations of different random_states.

random = [47, 103, 333, 253, 66, 31]

acc_list = []
# roc_list = []
# cp_list = []
# cpMean_list = []
# cpSTD_list = []

# Run through the random numbers.
for state in random:
  
  # Create the Train/Test Split for each random number.
  Xcat_train, Xcat_test, ycat_train, ycat_test = cuml.model_selection.train_test_split(X, y_cat, test_size=0.2, shuffle=True, random_state=state, stratify=y_cat)

  RFclass_model = cuml.ensemble.RandomForestClassifier(max_depth=100, n_estimators=300, split_criterion=1)
  RFclass_model.fit(Xcat_train, ycat_train)
  y_pred = RFclass_model.predict(Xcat_test)
  y_pred = y_pred.astype(int)

#   RF_acc, RF_roc, RF_cp, RF_cpMean, RF_cpSTD = class_metric(ycat_test, y_pred)

  accuracy = str(np.round(cuml.metrics.accuracy.accuracy_score(ycat_test, y_pred), 3) * 100)
  print(f'Random State: {state}, Accuracy: {accuracy}%')

  acc_list.append(accuracy)
#   roc_list.append(RF_roc)
#   cp_list.append(RF_cp)
#   cpMean_list.append(RF_cpMean)
#   cpSTD_list.append(RF_cpSTD)

  del RFclass_model

Random State: 47, Accuracy: 68.30000000000001%


In [ ]:
# Print Accuracy Score for each Random State
print(f'Accuracy List: {acc_list}')
print(f'Accuracy Mean: {np.mean(acc_list)}')
print(f'Accurcay Mean STD: {np.std(acc_list)}')

### Random Forest Classifier, max_depth=100, n_estimators=300, Train/Test Split random=47

In [ ]:
# Create first train/test split for classification.  Automated folds from cross-validation are consuming too many resources and taking too long to process.
Xcat_train, Xcat_test, ycat_train, ycat_test = cuml.model_selection.train_test_split(X, y_cat, test_size=0.2, shuffle=True, random_state=47, stratify=y_cat)

In [ ]:
RFclass_model = cuml.ensemble.RandomForestClassifier(max_depth=100, n_estimators=300)
RFclass_model.fit(Xcat_train, ycat_train)
y_pred = RFclass_model.predict(Xcat_test)
y_pred = y_pred.astype(int)

In [ ]:
# Accuracy Assessment for Classification Model.
RF_acc, RF_roc, RF_cp, RF_cpMean, RF_cpSTD = class_metric(ycat_test, y_pred)

### Random Forest Classifier, max_depth=100, n_estimators=300, Train/Test Split random=103

In [ ]:
# Create first train/test split for classification.  Automated folds from cross-validation are consuming too many resources and taking too long to process.
Xcat_train, Xcat_test, ycat_train, ycat_test = cuml.model_selection.train_test_split(X, y_cat, test_size=0.2, shuffle=True, random_state=103, stratify=y_cat)

In [ ]:
RFclass_model = cuml.ensemble.RandomForestClassifier(max_depth=100, n_estimators=300)
RFclass_model.fit(Xcat_train, ycat_train)
y_pred = RFclass_model.predict(Xcat_test)
y_pred = y_pred.astype(int)

In [ ]:
# Accuracy Assessment for Classification Model.
RF_acc, RF_roc, RF_cp, RF_cpMean, RF_cpSTD = class_metric(ycat_test, y_pred)

### Random Forest Classifier, max_depth=100, n_estimators=300, Train/Test Split random=33

In [ ]:
# Create first train/test split for classification.  Automated folds from cross-validation are consuming too many resources and taking too long to process.
Xcat_train, Xcat_test, ycat_train, ycat_test = cuml.model_selection.train_test_split(X, y_cat, test_size=0.2, shuffle=True, random_state=33, stratify=y_cat)

In [ ]:
RFclass_model = cuml.ensemble.RandomForestClassifier(max_depth=100, n_estimators=300)
RFclass_model.fit(Xcat_train, ycat_train)
y_pred = RFclass_model.predict(Xcat_test)
y_pred = y_pred.astype(int)

In [ ]:
# Accuracy Assessment for Classification Model.
RF_acc, RF_roc, RF_cp, RF_cpMean, RF_cpSTD = class_metric(ycat_test, y_pred)

### Random Forest Classifier, max_depth=100, n_estimators=300, Train/Test Split random=253

In [ ]:
# Create first train/test split for classification.  Automated folds from cross-validation are consuming too many resources and taking too long to process.
Xcat_train, Xcat_test, ycat_train, ycat_test = cuml.model_selection.train_test_split(X, y_cat, test_size=0.2, shuffle=True, random_state=253, stratify=y_cat)

In [ ]:
RFclass_model = cuml.ensemble.RandomForestClassifier(max_depth=100, n_estimators=300)
RFclass_model.fit(Xcat_train, ycat_train)
y_pred = RFclass_model.predict(Xcat_test)
y_pred = y_pred.astype(int)

In [ ]:
# Accuracy Assessment for Classification Model.
RF_acc, RF_roc, RF_cp, RF_cpMean, RF_cpSTD = class_metric(ycat_test, y_pred)

### Random Forest Classifier, max_depth=100, n_estimators=300, Train/Test Split random=66

In [ ]:
# Create first train/test split for classification.  Automated folds from cross-validation are consuming too many resources and taking too long to process.
Xcat_train, Xcat_test, ycat_train, ycat_test = cuml.model_selection.train_test_split(X, y_cat, test_size=0.2, shuffle=True, random_state=66, stratify=y_cat)

In [ ]:
RFclass_model = cuml.ensemble.RandomForestClassifier(max_depth=100, n_estimators=300)
RFclass_model.fit(Xcat_train, ycat_train)
y_pred = RFclass_model.predict(Xcat_test)
y_pred = y_pred.astype(int)

In [ ]:
# Accuracy Assessment for Classification Model.
RF_acc, RF_roc, RF_cp, RF_cpMean, RF_cpSTD = class_metric(ycat_test, y_pred)

### Random Forest Classifier, split_criterion=entropy, max_depth=100, n_estimators=300, Train/Test Split random=66

In [ ]:
RFclass_model = cuml.ensemble.RandomForestClassifier(max_depth=100, n_estimators=300, split_criterion=1)
RFclass_model.fit(Xcat_train, ycat_train)
y_pred = RFclass_model.predict(Xcat_test)
y_pred = y_pred.astype(int)

In [ ]:
# Accuracy Assessment for Classification Model.
RF_acc, RF_roc, RF_cp, RF_cpMean, RF_cpSTD = class_metric(ycat_test, y_pred)

### Random Forest Classifier, split_criterion=entropy, max_depth=100, n_estimators=300, Train/Test Split random=33

In [ ]:
# Create first train/test split for classification.  Automated folds from cross-validation are consuming too many resources and taking too long to process.
Xcat_train, Xcat_test, ycat_train, ycat_test = cuml.model_selection.train_test_split(X, y_cat, test_size=0.2, shuffle=True, random_state=33, stratify=y_cat)

In [ ]:
RFclass_model = cuml.ensemble.RandomForestClassifier(max_depth=200, n_estimators=300, split_criterion=1)
RFclass_model.fit(Xcat_train, ycat_train)
y_pred = RFclass_model.predict(Xcat_test)
y_pred = y_pred.astype(int)

In [ ]:
# Accuracy Assessment for Classification Model.
RF_acc, RF_roc, RF_cp, RF_cpMean, RF_cpSTD = class_metric(ycat_test, y_pred)

Changing the split_critereon from the default gini to entropy does not appear to increase accuracy in a signficant manner.

## **Summary of Validation.**

In [ ]:
# Mean and STD of accuracy score.
acc_list = [76.9, 77.0, 77.0, 76.9, 76.9]

# Print Accuracy Score for each Random State
# print(f'Accuracy List: {acc_list}')
print(f'Accuracy Mean: {str(np.round(np.mean(acc_list), 3))}%')
print(f'Accurcay Mean STD: {str(np.round(np.std(acc_list), 3))}%')

## KFold Cross Validation

In [ ]:
# CRASHES SYSTEM DUE TO EXCEEDING AVAILABLE RAM.

# KFold Stratified Cross validation.  Need to convert to Numpy array in order to use cross_val_score.
X = X.to_numpy()
y_cat = y_cat.to_numpy()

# Create 5-folds for cross validation
cv_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=33)

# Run the model with the appropriate data types.
RFclass_model = cuml.ensemble.RandomForestClassifier(max_depth=100, n_estimators=300, output_type='numpy')
cv_acc = cross_val_score(RFclass_model, X, y_cat, cv=cv_fold)
print(f'RFclass_model: CV accuracy = {cv_acc.mean()} (std={cv_acc.std()})')

# Shapley Values

In [ ]:
# Create train/test split for model explainability.
Xcat_train, Xcat_test, ycat_train, ycat_test = cuml.model_selection.train_test_split(X, y_cat, test_size=0.2, shuffle=True, random_state=33, stratify=y_cat)

In [ ]:
# STILL CRASHES SYSTEM DUE TO EXCEEDING AVAILABLE RAM EVEN WITH REDUCED FEATURES.

# RAPIDS SHAP model explainability. 
RFclass_model = cuml.ensemble.RandomForestClassifier(max_depth=200, n_estimators=300)
RFclass_model.fit(Xcat_train, ycat_train)

cu_explainer = cuml.explainer.KernelExplainer(
    model=RFclass_model.predict,
    data=Xcat_train,
    is_gpu_model=True,
    random_state=33,
    dtype=np.float32,
    output_type='cupy')

cu_shap_values = cu_explainer.shap_values(Xcat_test)
cu_shap_values  

# **Conclusions**

## **Regression Model Summary**.

### Regression Models with SELECT FEATURES.

In [ ]:
# Table Summarizing metrics from the different regression models.
metric = [['Metric', 'Dummy Reg', 'Linear Reg', 'Ridge Reg', 'Lasso Reg', 'ElasticNet', 'Nearest Neighbor', 'Random Forest'],
         ['R**2', Dum_r2, Lin_r2, Ridge_r2, Lasso_r2, EN_r2, NN_r2, RF_r2],
         ['MSE', Dum_mse, Lin_mse, Ridge_mse, Lasso_mse, EN_mse, NN_mse, RF_mse],
         ['RMSE', Dum_rmse, Lin_rmse, Ridge_rmse, Lasso_rmse, EN_rmse, NN_rmse, RF_rmse],
         ['MAE', Dum_mae, Lin_mae, Ridge_mae, Lasso_mae, EN_mae, NN_mae, RF_mae]]
table = tabulate.tabulate(metric, tablefmt='html')

display(HTML(table))

Metric,Dummy Reg,Linear Reg,Ridge Reg,Lasso Reg,ElasticNet,Nearest Neighbor,Random Forest
R**2,0.622,0.157,0.13,-0.0,0.074,0.188,0.622
MSE,0.567,1.262,1.303,1.497,1.387,1.216,0.567
RMSE,0.753,1.123,1.142,1.224,1.178,1.103,0.753
MAE,0.52,0.858,0.888,0.974,0.929,0.776,0.52


### Regresion Models with ALL FEATURES.

In [ ]:
# Table Summarizing metrics from the different regression models FOR ALL FEATURES.
metric = [['Metric', 'Dummy Reg', 'Linear Reg', 'Ridge Reg', 'Lasso Reg', 'ElasticNet', 'Nearest Neighbor', 'Random Forest'],
         ['R**2', Dum_r2, Lin_r2, Ridge_r2, Lasso_r2, EN_r2, NN_r2, RF_r2],
         ['MSE', Dum_mse, Lin_mse, Ridge_mse, Lasso_mse, EN_mse, NN_mse, RF_mse],
         ['RMSE', Dum_rmse, Lin_rmse, Ridge_rmse, Lasso_rmse, EN_rmse, NN_rmse, RF_rmse],
         ['MAE', Dum_mae, Lin_mae, Ridge_mae, Lasso_mae, EN_mae, NN_mae, RF_mae]]
table = tabulate.tabulate(metric, tablefmt='html')

display(HTML(table))

Metric,Dummy Reg,Linear Reg,Ridge Reg,Lasso Reg,ElasticNet,Nearest Neighbor,Random Forest
R**2,-0.0,0.215,0.154,-0.0,0.074,0.468,0.714
MSE,1.497,1.176,1.266,1.497,1.387,0.796,0.429
RMSE,1.224,1.084,1.125,1.224,1.178,0.892,0.655
MAE,0.974,0.819,0.872,0.974,0.929,0.574,0.434


## **Classification Model Summary.**

### Classification Models with SELECT FEATURES.

In [ ]:
metric = [['Metric','Logistic Regression', 'Nearest Neighbor', 'Random Forest'],
          ['ROC AUC', '0.631', '0.976', '1.253'],
          ['Total Accuracy', '69.8%', '60.6%', '68.8%'],
          ['Mean Accuracy per Class', '26.3', '35.8%', '65.4%'],
          ['Accuracy per class STD', '15.8%', '13.5%', '6.3%']]
table = tabulate.tabulate(metric, tablefmt='html')

display(HTML(table))

Metric,Logistic Regression,Nearest Neighbor,Random Forest
ROC AUC,0.631,0.976,1.253
Total Accuracy,69.8%,60.6%,68.8%
Mean Accuracy per Class,26.3,35.8%,65.4%
Accuracy per class STD,15.8%,13.5%,6.3%


### Classification Models with ALL FEATURES.

In [ ]:
metric = [['Metric','Logistic Regression', 'Nearest Neighbor', 'Random Forest'],
          ['ROC AUC', '0.844', '1.472', '1.72'],
          ['Total Accuracy', '60.0%', '68.2%', '77.0%'],
          ['Mean Accuracy per Class', '27.4%', '55.2%', '74.6%'],
          ['Accuracy per class STD', '16.2%', '9.2%', '5.3%']]
table = tabulate.tabulate(metric, tablefmt='html')

display(HTML(table))

Metric,Logistic Regression,Nearest Neighbor,Random Forest
ROC AUC,0.844,1.472,1.72
Total Accuracy,60.0%,68.2%,77.0%
Mean Accuracy per Class,27.4%,55.2%,74.6%
Accuracy per class STD,16.2%,9.2%,5.3%


## **Best Model: Random Forest (max_depth=100, n_estimators=300)  ALL FEATURES**

In [ ]:
# Mean and STD of accuracy score.
acc_list = [76.9, 77.0, 77.0, 76.9, 76.9]

# Print Accuracy Score for each Random State
# print(f'Accuracy List: {acc_list}')
print(f'Total Accuracy Mean: {str(np.round(np.mean(acc_list), 3))}%')
print(f'Total Accurcay Mean STD: {str(np.round(np.std(acc_list), 3))}%')

Total Accuracy Mean: 76.94%
Total Accurcay Mean STD: 0.049%


In [ ]:
# Accuracy for each Drought Score Category.
metric = [['Class', 'Accuracy'],
          ['0', '78.8%'],
          ['1', '73.5%'],
          ['2', '68.9%'],
          ['3', '68.7%'],
          ['4', '74.3%'],
          ['5', '83.7%']]
table = tabulate.tabulate(metric, tablefmt='html')

display(HTML(table))

Class,Accuracy
0,78.8%
1,73.5%
2,68.9%
3,68.7%
4,74.3%
5,83.7%


**Conclusion:** 
The regression models do not give a good enough accuracy for practical implementation since the best Mean Absolute Error is 0.434 and a +/- of 0.434 does not give a good enough separation between the underlying Drought Score Categories.  Therefore, the modeling switched to classification.  The Random Forest Classification model gives a much more practical implementation for this project with an overall accuracy of 77% and a mean per class accuracy of 74.6 +/- 5.3%.

All of the models were rerun with a selection of features instead of using all features due to several features being highly correlated.  A single feature was chosen to represent groups of features that had a correlation of higher than 0.92.  This was primarily done to determine the influence and impact of the variables on model performance, since this is not possible to determine when the variables are highly correlated.  There is some insight with the regression models on which variables are important for the model.  Unfortunately, Shapley values still could not be determined for the Random Forest model.  The models rerun with select features all preformed significantly worse and provided no insight into how the variables were influencing the best model - Random Forest.